# I. **Initial Set Up**

## **Project on Evaluating Machine Learning Models.**
 
**Instructions.** Select a dataset from [UCI](https://archive.ics.uci.edu/ml/datasets.php) or [Google](https://datasetsearch.research.google.com/), formulate a machine learning problem (supervised or unsupervised), and build and evaluate two models (different methods) that solve the problem. Any programming language may be used. 
- You may also use other legitimate sources at the same level of the UCI and Google sites provided. 
- You may use methods not taught in class. KNN is not an option. 
- You may also use a portion of the dataset if its size causes problems (e.g. reduce the number of rows)

**Deliverables.** In a Google Drive folder that I can access, submit the following: 
- Source code and executables
- Instructions on how to use your resources (i.e. your program)
- Slide deck explaining your work
- Recorded video presentation of your work (approx 20-30mins)

**Expected Output.**
- Jupyter Notebook (.ipynb)
- Resources (csv unclean and cleaned)
- Video Presentation
- Slide Deck Presentation

# **II. Data Set**

**Dataset Overview.**

The dataset contains raw information sourced from the Zomato Recommendation Platform for restaurants based in Pune, India, covering the year 2023. Each row corresponds to a single restaurant entry and includes a variety of attributes such as the restaurant’s name, multiple types of cuisine offered (up to eight slots), its categorized food type, the average cost for two people, the locality within Pune, and the average customer dining rating.

This dataset provides a foundation for predictive modeling and exploratory analysis, as it blends both categorical (e.g., cuisine types, locality) and numerical (e.g., rating, pricing) data. Through this structure, we can investigate patterns in consumer preferences, identify key factors influencing restaurant ratings, and evaluate the performance of machine learning models like Decision Trees and Mixed Naive Bayes in classifying highly rated restaurants.

| **Features**              | **Short Explanation**                                                         | **Possible Values / Example**                 |
| ------------------------ | ----------------------------------------------------------------------------- | --------------------------------------------- |
| `Restaurant_Name`        | Name of the restaurant listed on Zomato                                       | `"Le Plaisir"`, `"Savya Rasa"`                |
| `Cuisine1` to `Cuisine8` | Different types of cuisines offered by the restaurant, in order of prominence | `"South Indian"`, `"Desserts"`, `"MISSING"`   |
| `Category`               | Grouped categories combining all cuisine types into a readable list           | `"Cafe, Italian, Continental..."`             |
| `Pricing_for_2`          | Approximate cost for two people, in INR                                       | `600`, `1200`, `2100`                         |
| `Locality in Pune`       | Location/neighborhood of the restaurant in Pune                               | `"Koregaon Park"`, `"Baner"`, `"Viman Nagar"` |
| `Dining_Rating`          | Average customer rating of the restaurant (out of 5)                          | `4.2`, `3.8`, `4.9`                           |


# **III. Ideal Pipeline**

Our goal for this analysis is to be able to determine which model is able to more accurately predict what are the top restaurants in the locale (possibly depending on cuisines, locality, or average price.) <!-- Expound >

**1. Data Preprocessing**
- Load and Inspection of data.
- Cleaning the data (i.e. Tableau) <!-- care of Godwyn -->

**2. Exploratory Data Analysis (EDA)**
- This will be more on understanding which features create a reactive effect towards the rest of the feature. 
- Identifies which feature is able to change the course of the data. From there, we will implement the models.

**3. Decision Tree Implementation 1 (DT1)**
- This will be one of the initial basis of our model apart from EDA.

**4. Apply Decision Tree Implementation 2 (DT2)**
- The second implementation of Decision Tree will consist of the data set where we have omitted certain features (To be identified soon. _i.e., MISSING values, certain irrelevant features_) based on our domain knowledge.
- Comparing this to Decision Tree Implementation 1, we may be able to justify that omitting certain "junk" features can make Decision Tree model more accurate.

**5. Apply Mixed Naive Bayes (MNB)**
- The final model we use in this study is the Mixed Naive Bayes (MNB) classifier. This model is a variation of the standard Naive Bayes algorithm that allows us to handle both categorical and continuous features—whic makes it especially well-suited for real-world datasets like Zomato’s, where variables such as cuisine type (categorical) and average price (numerical) coexist.

**6. Conclusion**
- Generally, through ***Exploratory Data Analysis (EDA) and both Decision Tree implementations***, you may conclude that certain features—such as Cuisine type, Locality, or Average Price—have a strong influence on whether a restaurant receives high ratings. _Features like 'MISSING' or non-informative columns could be confirmed as noise, negatively affecting model accuracy._
- Comparing ***Decision Tree 1 (all features) with Decision Tree 2 (cleaned features)***, you might find that:
    - Removing irrelevant or noisy features leads to higher accuracy and simpler tree structures.
    - This supports the idea that domain knowledge-based feature pruning improves model performance.
- ***Mixed Naive Bayes (MNB) might perform competitively or better on some metrics*** (like precision or recall) compared to Decision Trees, especially in cases where feature independence is mostly true. However, MNB might underperform if features are highly correlated, where Decision Trees can better handle interactions.

# **IV. Data Preprocessing**

< This section will include general importing and inspection of the data. Cleaning the data as well for nullified or duplicated values. > <!-- Expound more >

In [ ]:
## Assume that we do not have the necessary libraries installed. 
%pip install pandas numpy matplotlib seaborn scikit-learn mixed-naive-bayes #This is to install the libraries needed to run the code.
%pip install --upgrade pip #Updates pip

# Need to install tkinter.
# For mac: brew install python-tk

import pandas as pd
import numpy as np
# import tkinter as tk
# from tkinter import filedialog

import math

from sklearn.tree import DecisionTreeClassifier, plot_tree
import matplotlib.pyplot as plt
from scipy.stats import percentileofscore
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, KFold ## https://www.geeksforgeeks.org/cross-validation-machine-learning/
from mixed_naive_bayes import MixedNB

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error



## Use if you are using Google Drive
import io

## Use if you are using Google Colab
# from google.colab import files
# uploaded = files.upload()

In [ ]:
## If using Jupyter Notebook / Run Locally via VS Code. Import the file LOZADA, BINWAG, CSCI 211 Zomato Dataset Pune.csv
# Hardcoded file path to your dataset
file_path = "LOZADA, BINWAG, CSCI 211 Zomato Dataset Pune.csv"
zomato_pune = pd.read_csv(file_path)

# Create a working copy for analysis
zomato_for_eda = zomato_pune.copy()

# Display the data
zomato_for_eda.head()

In [ ]:
# Display the data
zomato_for_eda.tail()

## **Restaurant Count per Locality**

In [ ]:
# Count restaurants per locality
locality_counts = zomato_for_eda['Locality in Pune'].value_counts()

# Plot the top 15
plt.figure(figsize=(12, 6))
sns.barplot(x=locality_counts.head(15).values, y=locality_counts.head(15).index, palette="viridis")

plt.title("Top 15 Localities by Number of Restaurants")
plt.xlabel("Number of Restaurants")
plt.ylabel("Locality")
plt.tight_layout()
plt.show()

## **Listing All Cuisines**

In [ ]:
# List of cuisine columns
cuisine_cols = [f'Cuisine{i}' for i in range(1, 9)]

# Flatten, drop NAs and "MISSING", then get unique values
all_cuisines = pd.unique(
    zomato_for_eda[cuisine_cols]
    .values
    .ravel()
)

# Clean list
unique_cuisines = sorted([c for c in all_cuisines if pd.notna(c) and c != 'MISSING'])

# Display
print("Number of unique cuisines:", len(unique_cuisines))
print(unique_cuisines)


## **Correlation between Pricing_for_2 and Dining_Rating.**
This is to be able to understand if pricing is "cheaper" gains a better rating as a restaurant. However, this is just a shallow experiment as Pricing can't be the only factor in a high-rating. 

In [ ]:
plt.hexbin(zomato_for_eda['Pricing_for_2'], zomato_for_eda['Dining_Rating'], gridsize=30, cmap='Blues')
plt.colorbar(label='Count in Bin')
plt.xlabel('Pricing for 2')
plt.ylabel('Dining Rating')
plt.title('Hexbin: Price vs Rating')
plt.show()

In [ ]:
correlation = zomato_for_eda[['Pricing_for_2', 'Dining_Rating']].corr()
print("Correlation between Pricing and Rating:")
print(correlation)

## **Correlation between Locality and Cuisine (1-8) to Dining_Rating.**
We proceed to test if there is a correlation between a cuisine served in certain locality. Such that, if for instance, `Mediterranean` and `European` cuisines served in	`Koregaon Park` receives a high rating whilst `Coffee` and `Desserts` served in the same locale has low-ratings.

In [ ]:
# Reshape cuisine columns into one
cuisine_cols = [f'Cuisine{i}' for i in range(1, 9)]

# Melt cuisine columns
long_df = zomato_for_eda.melt(
    id_vars=['Dining_Rating', 'Locality in Pune'],
    value_vars=cuisine_cols,
    var_name='CuisineCol',
    value_name='Cuisine'
)

# Drop missing cuisines
long_df = long_df.dropna(subset=['Cuisine'])

# Grouping
rating_by_combo = (
    long_df
    .groupby(['Cuisine', 'Locality in Pune'])['Dining_Rating']
    .mean()
    .reset_index()
    .rename(columns={'Dining_Rating': 'Avg_Rating'})
)

# Top 15 highest-rated cuisine-location combos.
# Maintains the original index of the row.
rating_by_combo.sort_values('Avg_Rating', ascending=False).head(15)

# Revises the index starting from 0.
# Uncomment if prefer to use original indexing.
    # cleaned_top = rating_by_combo.sort_values('Avg_Rating', ascending=False).head(15).reset_index(drop=True)
    # cleaned_top # Prints



In [ ]:
# Filter to threshold. Only Avg_Rating over 3.8
filtered = rating_by_combo[rating_by_combo['Avg_Rating'] > 3.8]

# Keep only top cuisines and localities by frequency
top_cuisines = (
    long_df['Cuisine'].value_counts()
    .loc[lambda x: x.index != 'MISSING']
    .head(10).index
)

top_localities = long_df['Locality in Pune'].value_counts().head(10).index

# Apply filter
filtered = filtered[
    (filtered['Cuisine'].isin(top_cuisines)) &
    (filtered['Locality in Pune'].isin(top_localities))
]

# Pivot
heatmap_data = filtered.pivot(
    index="Cuisine",
    columns="Locality in Pune",
    values="Avg_Rating"
)

# Plot
plt.figure(figsize=(10, 6))
sns.heatmap(heatmap_data, annot=True, fmt=".1f", cmap="YlGnBu", linewidths=0.5, linecolor='gray')
plt.title("Dining Rating > 3.8 — Top 10 Cuisines x Top 5 Localities")
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

## **Price Frequency**

In [ ]:
plt.figure(figsize=(10, 6))

# Filter the dataset
filtered = zomato_for_eda[zomato_for_eda['Dining_Rating'] >= 3.8]

# Plot histogram with KDE
sns.histplot(filtered['Pricing_for_2'], bins=20, kde=False, color='seagreen')
plt.title("Price Distribution for Restaurants with Rating ≥ 3.8")
plt.xlabel("Price for 2")
plt.ylabel("Number of Restaurants")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# Step 1: Define the features and filter the DataFrame
input_features = ["Cuisine1", "Cuisine2", "Cuisine3", "Cuisine4", 
                  "Cuisine5", "Cuisine6", "Cuisine7", "Cuisine8", 
                  "Pricing_for_2", "Locality in Pune"]

X = zomato_for_eda.filter(items=input_features)


In [ ]:
# Step 1: Flatten all cuisines
cuisine_cols = [f'Cuisine{i}' for i in range(1, 9)]
cuisines_flat = pd.Series(X[cuisine_cols].values.ravel())

# Step 2: Clean values
cuisines_flat = cuisines_flat.replace("MISSING", np.nan).dropna()

# Step 3: Count top 15
cuisines_freq = cuisines_flat.value_counts().head(15)

# Step 4: Plot
plt.figure(figsize=(10, 6))
sns.barplot(x=cuisines_freq.values, y=cuisines_freq.index)
plt.title("Top 15 Most Common Cuisines")
plt.xlabel("Number of Occurrences")
plt.tight_layout()
plt.show()


## **More EDA**

In [ ]:
# Set rating threshold
rating_threshold = 3.75

# First-level helper functions
def find_percent(df, feature_group, feature):
    feature_filter = df.loc[df[feature_group] == feature]
    if not feature_filter.empty:
        percent_above_cutoff = 100 - percentileofscore(
            feature_filter['Dining_Rating'], rating_threshold, kind='strict'
        )
    else:
        percent_above_cutoff = 0
    return percent_above_cutoff

def find_mean_rating(df, feature_group, feature):
    feature_filter = df.loc[df[feature_group] == feature]
    return feature_filter['Dining_Rating'].mean() if not feature_filter.empty else np.nan

# Second-level EDA helper
def eda_resto_data_numerical(df, feature_group):
    feature_list = pd.unique(df[feature_group]).tolist()
    feature_list2 = [feature for feature in feature_list if feature != 'MISSING' and pd.notnull(feature)]

    percent_exceeding_cutoff_feature = []
    average_per_cutoff_feature = []

    for feature in feature_list2:
        above_threshold = round(find_percent(df, feature_group, feature), 4)
        percent_exceeding_cutoff_feature.append(above_threshold)

        mean_rating = round(find_mean_rating(df, feature_group, feature), 4)
        average_per_cutoff_feature.append(mean_rating)

    # MinMax scaling
    scaler = MinMaxScaler()
    mean_scaled = scaler.fit_transform(np.array(average_per_cutoff_feature).reshape(-1, 1)).flatten()
    mean_scaled = [round(score, 4) for score in mean_scaled]

    # Create summary DataFrame
    df_mean_feature = pd.DataFrame({
        feature_group: feature_list2,
        '% Above 3.75': percent_exceeding_cutoff_feature,
        'Mean Rating': average_per_cutoff_feature,
        'MinMax Scale Score': mean_scaled
    })

    df_mean_feature = df_mean_feature.sort_values(by=['Mean Rating', feature_group], ascending=[False, True]).reset_index(drop=True)
    df_mean_feature['Rank'] = df_mean_feature.index + 1

    return df_mean_feature

In [ ]:
eda_locality = eda_resto_data_numerical(zomato_for_eda, 'Locality in Pune')
eda_locality.head()

In [ ]:
eda_resto_cuisine1 = eda_resto_data_numerical(zomato_for_eda, 'Cuisine1')
eda_resto_cuisine1.head()

In [ ]:
eda_resto_cuisine2 = eda_resto_data_numerical(zomato_for_eda, 'Cuisine2')
eda_resto_cuisine2.head()

In [ ]:
eda_resto_cuisine3 = eda_resto_data_numerical(zomato_for_eda, 'Cuisine3')
eda_resto_cuisine3.head()

In [ ]:
eda_resto_cuisine4 = eda_resto_data_numerical(zomato_for_eda, 'Cuisine4')
eda_resto_cuisine4.head()

In [ ]:
eda_resto_cuisine5 = eda_resto_data_numerical(zomato_for_eda, 'Cuisine5')
eda_resto_cuisine5.head()

In [ ]:
eda_resto_cuisine6 = eda_resto_data_numerical(zomato_for_eda, 'Cuisine6')
eda_resto_cuisine6.head()

In [ ]:
eda_resto_cuisine7 = eda_resto_data_numerical(zomato_for_eda, 'Cuisine7')
eda_resto_cuisine7.head()

In [ ]:
eda_resto_cuisine8 = eda_resto_data_numerical(zomato_for_eda, 'Cuisine8')
eda_resto_cuisine8.head()

In [ ]:
zomato_pune_yes = zomato_for_eda[zomato_for_eda['Dining_Rating'] >= 3.8]
zomato_pune_yes.head()

In [ ]:
zomato_pune_no = zomato_for_eda[zomato_for_eda['Dining_Rating'] < 3.8]
zomato_pune_no.head()

In [ ]:
min_cost = min(zomato_pune_yes['Pricing_for_2'])
max_cost = max(zomato_pune_yes['Pricing_for_2'])

num_bins = int((max_cost - min_cost) / 50) + 1
num_bins

plt.figure(figsize=(15, 5))
sns.histplot(zomato_pune_yes['Pricing_for_2'], bins = num_bins, color='magenta')
plt.title("Meal Cost Distribution of the Zomato Dataset in Pune (if rating >= 3.8)")
plt.xlabel("Cost of Meal for 2 (INR)")
plt.xlim(0, 5000)
plt.ylabel("Frequency")
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

In [ ]:
min_cost = min(zomato_pune_no['Pricing_for_2'])
max_cost = max(zomato_pune_no['Pricing_for_2'])

num_bins = int((max_cost - min_cost) / 50) + 1
num_bins

plt.figure(figsize=(15, 5))
sns.histplot(zomato_pune_no['Pricing_for_2'], bins = num_bins, color='red')
plt.title("Meal Cost Distribution of the Zomato Dataset in Pune (if rating < 3.8)")
plt.xlabel("Cost of Meal for 2 (INR)")
plt.xlim(0, 5000)
plt.ylabel("Frequency")
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

In [ ]:
## Map the cuisines to numerical features according

def feature_map(feature_df, row_feature):
    feature_map = {'MISSING' : 0}

    for index, row in feature_df.iterrows():
        feature_name = row[row_feature]
        rank = row['Rank']

        feature_map[feature_name] = rank
    return feature_map

# Create mappings for cuisine 1 to cuisine 8
cuisine1_map = feature_map(eda_resto_cuisine1, 'Cuisine1')
cuisine2_map = feature_map(eda_resto_cuisine2, 'Cuisine2')
cuisine3_map = feature_map(eda_resto_cuisine3, 'Cuisine3')
cuisine4_map = feature_map(eda_resto_cuisine4, 'Cuisine4')
cuisine5_map = feature_map(eda_resto_cuisine5, 'Cuisine5')
cuisine6_map = feature_map(eda_resto_cuisine6, 'Cuisine6')
cuisine7_map = feature_map(eda_resto_cuisine7, 'Cuisine7')
cuisine8_map = feature_map(eda_resto_cuisine8, 'Cuisine8')

# Create mapping for locality
locality_map = feature_map(eda_locality, 'Locality in Pune')

# map the columns for cuisine
zomato_for_eda['Cuisine1'] = zomato_for_eda['Cuisine1'].map(cuisine1_map)
zomato_for_eda['Cuisine2'] = zomato_for_eda['Cuisine2'].map(cuisine2_map)
zomato_for_eda['Cuisine3'] = zomato_for_eda['Cuisine3'].map(cuisine3_map)
zomato_for_eda['Cuisine4'] = zomato_for_eda['Cuisine4'].map(cuisine4_map)
zomato_for_eda['Cuisine5'] = zomato_for_eda['Cuisine5'].map(cuisine5_map)
zomato_for_eda['Cuisine6'] = zomato_for_eda['Cuisine6'].map(cuisine6_map)
zomato_for_eda['Cuisine7'] = zomato_for_eda['Cuisine7'].map(cuisine7_map)
zomato_for_eda['Cuisine8'] = zomato_for_eda['Cuisine8'].map(cuisine8_map)

# Map the columns for locality in Pune
zomato_for_eda['Locality in Pune'] = zomato_for_eda['Locality in Pune'].map(locality_map)

# Create binary classification column
zomato_for_eda['isHighRating'] = (zomato_for_eda['Dining_Rating'] >= rating_threshold).astype(int)

# Drop the original numerical rating column
zomato_for_eda = zomato_for_eda.drop(columns=['Dining_Rating'])

# Reorder columns to ensure binary classification is at the rightmost position
zomato_for_eda = zomato_for_eda[[col for col in zomato_for_eda.columns if col not in ['isHighRating']] + ['isHighRating']]

In [ ]:
input_features = ["Cuisine1", "Cuisine2", "Cuisine3", "Cuisine4", "Cuisine5", "Cuisine6", "Cuisine7", "Cuisine8", "Pricing_for_2", "Locality in Pune"]

X = zomato_for_eda.filter(items = input_features)
X

In [ ]:
y = zomato_for_eda["isHighRating"]
y

## **Post-EDA**


After reviewing the data, we've understood the following.
- 

# **V. Decision Tree Implementation 1 (DT1)** 

In [ ]:
## 70% train, 30% others (as a working concept, assume that other)
X_train, X_other, y_train, y_other = train_test_split(X, y, test_size = 0.3, random_state = 45)

In [ ]:
dec_tree = DecisionTreeClassifier(random_state = 45)
dec_tree.fit(X_train, y_train)

y_pred = dec_tree.predict(X_other)
cm = confusion_matrix(y_other, y_pred)
cm_display = ConfusionMatrixDisplay(cm, display_labels = dec_tree.classes_)
cm_display.plot(cmap = plt.cm.Blues)
plt.show()

In [ ]:
tn, fp, fn, tp = cm.ravel()
sensitivity = tp / (tp + fn) if (tp + fn) != 0 else 0
specificity = tn / (tn + fp) if (tn + fp) != 0 else 0
accuracy = accuracy_score(y_other, y_pred)
print(f"True Positives: {tp}")
print(f"Sensitivity: {sensitivity*100:.2f} %")
print(f"Specificity: {specificity*100:.2f} %")
print(f"Accuracy: {accuracy*100:.2f} %")


The current Decisition Tree (DT1) implementation consists of the basic cleaned data. This means that we've decided to keep the `MISSING` values that occur in the way the data is collected. While restaurants may have 1-2 cuisines, some have over 2 cuisines served. This is the reason why there is `cuisine 1 to 8` in the dataset. As such, `MISSING` values may incur false-negative results in predicting `Dining_Ratings`. In this case, DT1 has an accuracy of `62.08%`.

## **VI. Decision Tree Implementation 2 (DT2)**

For DT2, the dataset has been further cleaned by removing ` MISSING ` values and making sure that there are no _"Junk"_ data that possibly makes Decision Trees lose its accuracy.

In [ ]:
# Replace 'MISSING' (0) with -1
cuisine_cols = [f'Cuisine{i}' for i in range(1, 9)]
X_all = zomato_for_eda.drop(columns=['isHighRating']).copy()
X_all[cuisine_cols] = X_all[cuisine_cols].replace(0, -1)

# Keep only numeric columns (drop Restaurant_Name etc.)
X_clean = X_all.select_dtypes(include=[np.number])
y_clean = zomato_for_eda['isHighRating']

# Train-test split
X_train2, X_other2, y_train2, y_other2 = train_test_split(X_clean, y_clean, test_size=0.3, random_state=45)

# Train the model
dec_tree2 = DecisionTreeClassifier(random_state=45)
dec_tree2.fit(X_train2, y_train2)

# Predict and evaluate
y_pred2 = dec_tree2.predict(X_other2)
cm2 = confusion_matrix(y_other2, y_pred2)
cm_display2 = ConfusionMatrixDisplay(cm2, display_labels=dec_tree2.classes_)
cm_display2.plot(cmap=plt.cm.Greens)
plt.title("Confusion Matrix for Decision Tree 2")
plt.show()

# Metrics
tn2, fp2, fn2, tp2 = cm2.ravel()
sensitivity2 = tp2 / (tp2 + fn2) if (tp2 + fn2) != 0 else 0
specificity2 = tn2 / (tn2 + fp2) if (tn2 + fp2) != 0 else 0
accuracy2 = accuracy_score(y_other2, y_pred2)

print(f"📊 Decision Tree 2 Metrics (Ignoring MISSING features):")
print(f"True Positives: {tp2}")
print(f"Sensitivity: {sensitivity2*100:.2f} %")
print(f"Specificity: {specificity2*100:.2f} %")
print(f"Accuracy: {accuracy2*100:.2f} %")


In [ ]:

feature_importances = pd.Series(dec_tree2.feature_importances_, index=X_clean.columns)
feature_importances = feature_importances.sort_values(ascending=False)
print(feature_importances.head(10))

## **VII. Mixed Naive Bayes (MNB)**

Implementing MNB to the cleaned dataset (_same with cleaned dataset of DT2_). 

## **VIII. General Finding and Analysis**

Explaining what we've found. (i.e., DT1 showed an accuracy of 60%.. However, DT2 as predicted, was able to get an accuracy of 80%... And lastly, MNB was able to get an accuracy of...)

## **IX. Conclusion**

Based on the findings, it is evident that DT2 and MNB was able to accurately predict... as such, we think it would be more accurate to remove _"Junk"_ data such as `MISSING, NULL, -12931931, 13412131` that can affect the model in predicting. 

In the case of predicting ` dining ratings `, it is possible that there could be a better model to be used since **Decision Trees** and **Mixed Naive Bayes** are only able to get `x%` and `y%`. Judging from the dataset, it may be even viable to use **Ensemble Learning Models (random forest)**, or the need to heavily manipulate the data to accomodate ` MISSING ` values and so on...

## **X. References**